In [7]:
from random import choice, random
import string

target = list("print('hello world')")
alphabet = string.ascii_lowercase + " " + "'" + "(" + ")"

pmut = 0.05
nchildren = 100

def fitness(trial):
    return sum(t != h for t,h in zip(trial, target))

def mutate(parent):
    return [(choice(alphabet) if random() < pmut else ch) for ch in parent]

parent = [choice(alphabet) for _ in range(len(target))]
igen = 0
while parent != target:
    children = (mutate(parent) for _ in range(nchildren))
    parent = min(children, key=fitness)
    print(f"{igen} gen: {''.join(parent)}")
    igen += 1

0 gen: p(uszxysjuzequz'trzt
1 gen: p(iszxysaueequzq rzt
2 gen: p(isz(ysaueequzq rzt
3 gen: p(isz(ysaieequzr rzt
4 gen: p(isz(ysaleequzr rzt
5 gen: p(isz(ysaleequzr r't
6 gen: p(isz(ysaleeqfzr r't
7 gen: p(isz(yseleeqfzr r't
8 gen: p(isz(yselez fzr r't
9 gen: paisz(yselez fzrlr'u
10 gen: paisz(yselez fzrlr')
11 gen: paihz(yhelez fzrlr')
12 gen: paihz(yhelez forlr')
13 gen: paihz(yhelez forlr')
14 gen: paihz(yhelez forlr')
15 gen: paihz(yhelez forlr')
16 gen: paihz(yhelez worlr')
17 gen: paihz(lhelez worlr')
18 gen: paihz(lhelez worlr')
19 gen: paiht(lhelez worlr')
20 gen: paiht(lhelez worlr')
21 gen: paiht(lhelez worlr')
22 gen: paint(lhelez worlr')
23 gen: paint(lhele' world')
24 gen: paint(lhele' world')
25 gen: paint(lhele' world')
26 gen: paint(lhele' world')
27 gen: paint('hele' world')
28 gen: paint('hele' world')
29 gen: pbint('hele' world')
30 gen: pbint('hele' world')
31 gen: pbint('hele' world')
32 gen: pbint('hele' world')
33 gen: pbint('hele' world')
34 gen: pbint('hele' wor

### Geneticke programovani

In [ ]:
import operator
import math
import random

import numpy

!pip install deap
from deap import algorithms #obsahuje algoritmz EA, GA atd.
from deap import base       #obsahuje nastroje pro prirazeni funkci do operatoru, kompilaci, inicializace populace
from deap import creator    #propojuje stromovou strukturu s fitness funkci
from deap import tools      #obsahuje nastroje pro mereni statistik
from deap import gp         #nastroje pro vytvoreni jedince jako stromu do GP



#chranene deleni proti cislo/0
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

#vytvoreni stromu, koren stromu (MAIN), ma aritu = 1 (jeden vstup do funkce)
pset = gp.PrimitiveSet("MAIN", 1)

#pridani primitiv pro soucet, rozdil, soucin, chranene deleni (binarni operace)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)

#pridani primitiv pro negaci (zaporne cislo), cos a sin (unarni operace)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

#nahodne cislo -1, 0, 1, ktere se meni, kdykoliv ho pridame do stromu
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

#defaultni nazev vstupu funkce prejmenujeme na x
pset.renameArguments(ARG0='x')



#nastaveni fitness fce: vahy zaporne, protoze minimalizujeme odchylku phi(x) - f(x)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

#propojeni stromu s fitness funkci, jedinci jsou obecna class Individual
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)



#nastaveni procesu genetickeho programovani
toolbox = base.Toolbox()

#jedinec bude tvoren ze stromu (genHalfAndHalf) o velikosti 1 nebo dva uzly
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)

#tento strom expr je muj jedinec
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

#populace je vytvarena z jedincu
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#dej to cele dohromady (prevedeni do spustitelne formy)
toolbox.register("compile", gp.compile, pset=pset)



#optimalizujeme prolozeni 20 bodu kvarticke funkce f(x) = x**4 + x**3 + x**2 + x 
def evalSymbReg(individual, points):
    #premena stromu na volatelnou aproximacni funkci
    func = toolbox.compile(expr=individual)
    #chyba (phi(x)-f(x))**2
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    #vratime chybu souctu ctvercu, pozor evalute fce musi vracet n-tici, jinak nedefinovane problemy
    return math.fsum(sqerrors) / len(points),

#- jak se pocita fitness funkce
toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])

#- zaregistruj pod terminem select operator selekce a nastav velikost turnaje 3 
#(https://www.geeksforgeeks.org/tournament-selection-ga/)
toolbox.register("select", tools.selTournament, tournsize=3)

#- nastaveni reprodukcniho operatoru, jednobodove krizeni s uniformacni sanci, ze uzel stromu bude co-point
toolbox.register("mate", gp.cxOnePoint)

#nastaveni podstromu pro mutaci, jedna se o cely podstrom
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)

#nastaveni mutacniho operatoru, mututuje se tak, cely podstrom expr_mut se appenduje do uzlu
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

#zavedeni hloubkovych limitu operatoru, Koza doporucuje 17
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))



def main():

    #pocatecni populace
    pop = toolbox.population(n=300)

    #struktura obsahujici nejlepsi kandidaty
    hof = tools.HallOfFame(1)
    
    #jake vsechny statistiky chceme pocitat (chceme pocitat fitness a velikost jedince)
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)

    #na techto statistickach chceme spocitat prumer, s.odch., min a max
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    print(hof.items[0])

if __name__ == "__main__":
    main()

In [13]:
from random import random, sample, randint, choice
from string import ascii_lowercase as abeceda

def nahodni_jedinci(cil, pocet_jedincu):
    return ["".join([choice(abeceda+"()") for i in range(len(cil))]) for i in range(pocet_jedincu)]

def pocet_spravnych_symbolu(cil, populace):
    return [sum([t == l for t, l in zip(cil, populace[i])]) for i in range(len(populace))]

def selekce(populace, pocet_rodicu, fitness):
    return [rodic[0] for rodic in sorted(zip(populace, fitness), key=lambda x:x[1], reverse=True)[:pocet_rodicu]]

def krizeni(rodice, pocet_deti, pkrizeni):
    pary = [(sample(rodice, 2)) for i in range(pocet_deti)]
    return ["".join([p1[i] if random() < pkrizeni else p2[i] for i in range(len(p1))]) for p1, p2 in pary]

def mutace(deti, pmut):
    return ["".join([choice(abeceda+"()") if random()<pmut else l for l in deti[i]]) for i in range(len(deti))]

def evolucni_algoritmus(cil, pocet_rodicu, pocet_deti, pmut, pkrizeni, tmax):
    populace = nahodni_jedinci(cil, pocet_rodicu)
    for generace in range(tmax):
        fitness = pocet_spravnych_symbolu(cil, populace)
        populace = selekce(populace, pocet_rodicu, fitness)
        populace = krizeni(populace, pocet_deti, pkrizeni)
        populace = mutace(populace, pmut)
        print(generace, max(fitness))
    print(populace)

def main():
    evolucni_algoritmus(cil="print(hello world)",pocet_rodicu=2,pocet_deti=100,pmut=0.05,pkrizeni=0.5,tmax=10000)

if __name__ == "__main__":
    main()

0 1
1 2
2 2
3 3
4 5
5 5
6 5
7 6
8 7
9 7
10 8
11 9
12 10
13 10
14 10
15 11
16 11
17 11
18 11
19 11
20 11
21 11
22 11
23 11
24 12
25 12
26 12
27 12
28 12
29 12
30 13
31 13
32 13
33 14
34 14
35 15
36 15
37 15
38 15
39 15
40 15
41 15
42 15
43 15
44 15
45 15
46 15
47 15
48 15
49 15
50 15
51 15
52 15
53 15
54 15
55 15
56 15
57 15
58 15
59 15
60 15
61 15
62 15
63 15
64 15
65 15
66 15
67 15
68 15
69 15
70 15
71 15
72 15
73 15
74 15
75 15
76 15
77 15
78 15
79 15
80 15
81 15
82 15
83 15
84 15
85 15
86 15
87 15
88 15
89 15
90 15
91 15
92 15
93 15
94 15
95 15
96 15
97 15
98 15
99 15
100 15
101 15
102 15
103 15
104 15
105 15
106 15
107 15
108 15
109 15
110 15
111 15
112 15
113 15
114 15
115 15
116 15
117 15
118 15
119 15
120 15
121 15
122 15
123 15
124 15
125 15
126 15
127 15
128 15
129 15
130 15
131 15
132 15
133 15
134 15
135 15
136 15
137 15
138 15
139 15
140 15
141 15
142 15
143 15
144 15
145 15
146 15
147 15
148 15
149 15
150 15
151 15
152 15
153 15
154 15
155 15
156 15
157 15
158 15
159 15
16